In [2]:
import pickle
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr
import sounddevice as sd



code

In [ ]:
with open('data/aiml.json','r',encoding = 'utf-8') as f:
    aiml_data = json.load(f)
aiml_question = []
aiml_answer = []
for intent in aiml_data:
    tag = intent['tag']
    for pattern in intent['patterns']:
        aiml_question.append(pattern)
        aiml_answer.append(intent['responses'][0])
aiml_df = pd.DataFrame({
    'question' : aiml_question,
    'answer': aiml_answer
})

In [ ]:
faq_df = pd.read_csv(r'data\faq_dataset.csv', encoding='latin1',
    engine='python')
faq_df= faq_df[['question','answer']].dropna()
faq_df.dropna(inplace=True)
faq_df.drop_duplicates(inplace=True)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x83 in position 253: invalid start byte

In [7]:
python_df = pd.read_csv(r"data\python_interview_questions_500.csv")
python_df = python_df[['question','answer']]
python_df['question'] = python_df['question'].str.strip()
python_df ['answer'] =python_df['answer'].str.strip()
python_text = python_df['question'].tolist()


In [2]:
with open("data/intent.json") as file:
    data = json.load(file)
patterns =[]
tags = []

In [3]:
with open("data/content.json") as file:
    concepts = json.load(file)
concept_names = list(concepts.keys())


In [4]:
for intent in data["intents"]:
    tag = intent["tag"]
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(tag)

In [5]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(tags)

In [6]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(patterns)
concept_vectors = vectorizer.transform(concept_names)


In [7]:
print(X)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 363 stored elements and shape (116, 229)>
  Coords	Values
  (0, 114)	0.6980081742163726
  (0, 107)	0.5512047462653549
  (0, 15)	0.45711914904286344
  (1, 107)	0.7024574172468293
  (1, 193)	0.48344205628287634
  (1, 223)	0.2738888581478203
  (1, 216)	0.4447721310450792
  (2, 223)	0.3718737209630081
  (2, 27)	0.6563954355634745
  (2, 162)	0.6563954355634745
  (3, 107)	0.48993143475355494
  (3, 62)	0.674356777904597
  (3, 198)	0.5524582566442945
  (4, 44)	0.6035480105368504
  (4, 211)	0.6035480105368504
  (4, 3)	0.5210178480186836
  (5, 158)	0.6596892464030526
  (5, 79)	0.6596892464030526
  (5, 68)	0.3600280494077443
  (6, 3)	0.6039485799424053
  (6, 194)	0.6996151192842118
  (6, 117)	0.3818177545040176
  (7, 84)	0.5927211921779569
  (7, 21)	0.5927211921779569
  (7, 128)	0.54531016558128
  :	:
  (109, 7)	0.628280516816177
  (109, 157)	0.6829052536596455
  (110, 15)	0.4053418080294571
  (110, 61)	0.618945620520646
  (110, 154)	0

In [8]:
print(y)


[ 8  8  8  8  8  8  8  8  8  8  8 16 16 16 16 16 16 16 16 16 12 12 12 12
 12 12 12 12  4  4  4  4  4  4  2  2  2  2  2  2 15 15 15 15 15 15 15  3
  3  3  3  3  3  3 11 11 11 11 11 11 11  1  1  1  1  1 17 17 17 17 17 17
  5  5  5  5  5  5 10 10 10 10 10 10 10 13 13 13 13  0  0  0  0  0  9  9
  9  9  9  9  9 14 14 14 14 14  6  6  6  6  6  7  7  7  7  7]


In [9]:
model = LogisticRegression(max_iter=1000)
model.fit(X,y)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [10]:
import os

os.makedirs("model", exist_ok=True)

pickle.dump(model, open("model/chatbot_model.pkl", "wb"))
pickle.dump(vectorizer, open("model/vectorizer.pkl", "wb"))
pickle.dump(label_encoder, open("model/label_encoder.pkl", "wb"))

print("✅ Model files saved inside /model folder")

✅ Model files saved inside /model folder


In [11]:
# -------------------------------
# Semantic Search Function
# -------------------------------
def find_best_concept(user_text):
    user_vector = vectorizer.transform([user_text])
    similarity_score = cosine_similarity(user_vector, concept_vectors)

    best_index = similarity_score.argmax()
    best_score = similarity_score[0][best_index]

    if best_score < 0.15:
        return None

    return concept_names[best_index]


# -------------------------------
# Main Interaction Loop
# -------------------------------
while True:

    try:
        choice = int(input("\nSelect one:\n1. Voice\n2. Text\n> "))
    except:
        print("Invalid choice")
        continue

    # -------------------------------
    # Voice Input
    # -------------------------------
    if choice == 1:

        samplerate = 16000
        seconds = 5

        print("🎤 Speak now... or say exit")

        recording = sd.rec(int(seconds * samplerate),
                           samplerate=samplerate,
                           channels=1,
                           dtype='int16')
        sd.wait()

        audio_data = sr.AudioData(recording.tobytes(),
                                  samplerate,
                                  2)

        rec = sr.Recognizer()

        try:
            User_Input = rec.recognize_google(audio_data)
            print("You said:", User_Input)

        except sr.UnknownValueError:
            print("Could not understand voice")
            continue

    # -------------------------------
    # Text Input
    # -------------------------------
    else:
        User_Input = input("\nEnter a Question (type 'exit' to stop): ")

    if User_Input.lower() == "exit":
        break

    # -------------------------------
    # ML Topic Prediction
    # -------------------------------
    input_vector = vectorizer.transform([User_Input])
    prediction = model.predict(input_vector)
    topic = label_encoder.inverse_transform(prediction)[0]

    print("Detected Topic:", topic)

    # -------------------------------
    # Semantic Concept Search
    # -------------------------------
    best_concept = find_best_concept(User_Input)

    if best_concept:
        print("Answer:", concepts[best_concept])
    else:
        print("Answer: I understand the topic but don't have that concept yet.")

Detected Topic: machine_learning
Answer: Transfer learning reuses pretrained models.
Invalid choice
Invalid choice
Invalid choice
Invalid choice
Invalid choice
Invalid choice


In [ ]:

import numpy as np

samplerate = 16000
seconds = 5   # listening time

print("🎤 Speak now...")

# Record audio from mic (no file)
recording = sd.rec(int(seconds * samplerate),
                   samplerate=samplerate,
                   channels=1,
                   dtype='int16')
sd.wait()

# Convert numpy audio → AudioData (SpeechRecognition format)
audio_data = sr.AudioData(recording.tobytes(),
                          samplerate,
                          2)  # 2 bytes for int16

# Recognize speech
rec = sr.Recognizer()

try:
    text = rec.recognize_google(audio_data)
    print("You said:", text)
except sr.UnknownValueError:
    print("Could not understand")

🎤 Speak now...
